In [111]:
import random
import numpy as np

In [141]:
area = np.zeros((10,10),dtype=float)

AP_num = np.random.poisson(lam=3.0, size=None)
# MT_num = AP_num
MT_num = np.random.poisson(lam=5.0, size=None)
# AP_num = 2
# MT_num = 2
좌표 = []
area_dasi = area

## AP 발생
for k in range(AP_num) :
  x= np.random.randint(10,size=1) # AP의 위치
  y= np.random.randint(10,size=1)
  while True : # 겹치면 안됨. 
    if area_dasi[x,y] !=0 :
       x= np.random.randint(10,size=1) # AP의 위치
       y= np.random.randint(10,size=1)
       
    else : 
       break
  area_dasi[x,y] = 1 

## MT 발생
for k in range(MT_num) :
  x_dasi= np.random.randint(10,size=1) # MT의 위치
  y_dasi= np.random.randint(10,size=1)
  while True :
    if area_dasi[x_dasi,y_dasi]!=0 : # AP와 MT가 이미 있으면 다시 선언
      x_dasi= np.random.randint(10,size=1)
      y_dasi= np.random.randint(10,size=1)
    else :
      break
  area_dasi[x_dasi,y_dasi] = -1 

## 각 MT 마다 가장 가까운 AP를 구하는 식
for a in range(MT_num) :
  x_= np.where(area_dasi == -1)[0][a]
  y_= np.where(area_dasi == -1)[1][a]
  min = 10000
  x_좌표 = None
  y_좌표 = None
  for k in range(AP_num) : 
    i = np.where(area_dasi == 1)[0][k]
    j = np.where(area_dasi == 1)[1][k]
    cal = (i-x_)**2 + (j-y_)**2
    if min > cal :
      min = cal
      x_좌표 = i
      y_좌표 = j
  좌표.append([(x_,y_),(x_좌표,y_좌표)])

In [142]:
area_dasi

array([[ 0.,  0.,  1., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0., -1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [143]:
좌표

[[(0, 3), (0, 2)],
 [(2, 7), (3, 3)],
 [(3, 6), (3, 3)],
 [(5, 3), (3, 3)],
 [(5, 9), (7, 6)],
 [(7, 1), (3, 3)],
 [(8, 2), (7, 6)],
 [(9, 2), (7, 6)]]

In [145]:
G = np.zeros((MT_num, AP_num), dtype =float)
S = 10**0.8 # shadowing
X = np.random.exponential(scale=1.0, size=None) # rayleigh 
index =np.where(area_dasi == 1)
AP = []
for i in range(AP_num) :
  x_index = index[0][i]
  y_index = index[1][i]
  AP.append([x_index,y_index])

for i in range(MT_num) :
  y=0
  for k in AP :
    G[i,y] = S * X/(((좌표[i][0][0]*100 - k[0]*100)**2 + (좌표[i][0][1]*100 - k[1]*100)**2)**2)
    y+=1

In [146]:
G

array([[1.08029674e-08, 1.33369968e-10, 3.21134583e-12],
       [1.28453833e-11, 3.73805099e-11, 1.59807210e-11],
       [1.72847478e-11, 1.33369968e-10, 4.21990913e-11],
       [1.59807210e-11, 6.75185461e-10, 6.39228838e-11],
       [1.97278440e-12, 6.75185461e-12, 6.39228838e-11],
       [4.32118695e-12, 2.70074184e-11, 1.72847478e-11],
       [2.63744321e-12, 1.59807210e-11, 3.73805099e-11],
       [1.64654281e-12, 7.89113760e-12, 2.70074184e-11]])

In [147]:
AP

[[0, 2], [3, 3], [7, 6]]

In [148]:
AP_num

3

In [149]:
MT_num

8

Channel Gain 식은 s * x /d^4 P_tx로 나타 낼 수 있는데 여기서 s가 shadow fading factor을 의미한다.

모든 p가 (ex 1로) 동일하다고 가정하자

theta도 1로 가정

round robin 의 경우 각각 rate가 동일한 시간동안 흐름

반면 PF의 경우에는 datarate의 곱이 최대가 될 때을 사용

In [150]:
## round robin의 경우
SIR = 0
rates = []
SIRs = []
for i in range(MT_num) :
  Interference = 0
  count = 0
  for k in G[i] :
    if count == i :
      power = k
    else :
      Interference += 1 * k
    count +=1 
  if Interference == 0 :
    break 
  SIR = power/Interference
  SIRs.append(SIR)
  #print(SIR)
  rate = np.log10(1+SIR)
  #print(rate)
  rates.append(rate)
  
# total_rate = total_rate/AP_num #  down link이므로 AP의 갯수로 나누어줌. -> 사실 그이유보다 RR 은 시간당 돌아가면서 진행하기 때문에 이건 AP_num이 6이니 6단위 시간 당 한번씩 번갈아 간다는 의미이다.

In [151]:
inc=0
for k in AP :
  count = 0
  total_AP_rate = 0
  for i in range(MT_num) :
    if k[0] == 좌표[i][1][0] and k[1] == 좌표[i][1][1] :
      total_AP_rate+=rates[i]
      count += 1
  if count != 0 :
#    total_AP_rate/=count
    total_AP_rate/=count # time step
     #  down link이므로 AP에 걸리는 MT의 갯수로 나누어줌. 그래야 단위 시간 당 한 AP의 data rate를 알 수 있기 때문이다.
  inc+=1
  print('{}AP_RR_rate : {}'.format(inc,total_AP_rate))

1AP_RR_rate : 1.903608120767299
2AP_RR_rate : 0.19084195784199862
3AP_RR_rate : 0.25873687552176455


In [138]:
rates

[0.06339724961633574,
 0.19433987864562827,
 0.02124938155833566,
 2.527163975085636]

In [139]:
SIRs

[0.15717022306224376,
 0.5643714400181157,
 0.050145272342863506,
 335.6386488894773]

제일 SIR1/average date rate 가 큰 것을 골라주면 되는데 average date rate가 현재는 모두 0에 가까운 값이므로 SIR이 가장 높은 것을 초반에 선택해주고 시간 time에 대해서 따로 구해주면 되는데 여기서는 데이터량이 정해져 있지 않으므로 구하기 힘들다. 그렇기 때문에 모든 data를 더하고 그 값을 사용하는 것으로 생각해보자. 한 time step을 5단위로 나누어 보자.

In [152]:
# PF
inc=0
for k in AP :
  count = 0
  total_PF_rate = []
  SIRs_PF = []
  total_AP_rate = 0
  for i in range(MT_num) :
    if k[0] == 좌표[i][1][0] and k[1] == 좌표[i][1][1] :
      total_PF_rate.append(rates[i])
      SIRs_PF.append(SIRs[i])
      count += 1
  rates_re = np.zeros(len(total_PF_rate))+0.0001 # 0으로 나누면 안되기 때문에
  rates_re = rates_re.tolist()
  SIRs_PF_dasi = SIRs_PF
  rates_re_dasi = rates_re
  #for i in range(len(total_PF_rate)) :
  if count == 0 :
      pass
  else :
    #for i in range(1,count+1) : # 이러면 RR과 같다.
    for i in range(1,10+1) : # 10 step 시간 당
       for j in range(len(SIRs_PF_dasi)) :
          SIRs_PF_dasi[j] /= rates_re_dasi[j]
       k=SIRs_PF_dasi.index(max(SIRs_PF_dasi))
       rates_re[k] += total_PF_rate[k]
       rates_re_dasi = rates_re
       for j in range(len(rates_re_dasi)) :
          rates_re_dasi[j]/=i
       total_AP_rate += total_PF_rate[k]
    total_AP_rate = total_AP_rate/10
  inc+=1
  print('{}AP_PF_rate : {}'.format(inc,total_AP_rate))

1AP_PF_rate : 1.903608120767299
2AP_PF_rate : 0.12364716325002671
3AP_PF_rate : 0.2393037271341348


더 성능이 좋다. 하나 뿐이 MT가 없을 때는 RR이나 PF나 성능이 같다.